# Imports

In [1]:

import pandas
from pandasql import sqldf


# Configuration

In [2]:

xlsx_data_path = 'online_retail_II.xlsx'
xlsx_sheet_names = ['Year 2009-2010', 'Year 2010-2011']


# Read & Append Data

In [3]:

sheets = []

for sheet_id, sheet_name in enumerate(xlsx_sheet_names):
    print('Reading data from', sheet_name)
    
    sheet_data = pandas.read_excel(xlsx_data_path, sheet_name=sheet_name)
    sheet_data['sheet_id'] = sheet_id # include a feature that identifies which sheet the data is from
    
    sheets.append(sheet_data)
    
    del sheet_data
    
print('Finished Reading Data!')

# SQL column names cannot have spaces or special characters
data = pandas.concat(sheets).rename(columns={'Customer ID':'Customer_ID'}, inplace=False)

del sheets


Reading data from Year 2009-2010
Reading data from Year 2010-2011
Finished Reading Data!


# EDA with SQL

In [13]:

data.info() # print column names and null count of each variable


<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 0 to 541909
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer_ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
 8   sheet_id     1067371 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 81.4+ MB


## Complex Query Example and Purpose
The Question: We want to identify loyal customers (those who've made multiple purchases) and understand their **preferred product categories**, as well as how much they contribute to the **total quantity of items sold** within those categories.

This query helps us segment customers and discover insights like:

 - Which customers frequently buy from specific categories?
 - Are there high-volume buyers for certain product types?
 - How do different countries contribute to category sales in terms of quantity?

In [23]:

# canceled transactions are excluded

sqldf("""
SELECT
    c.Customer_ID,
    c.Country,
    c.Description, -- Now using Description as the product identifier
    SUM(c.Quantity) AS TotalQuantityBoughtOfProduct,
    COUNT(DISTINCT c.Invoice) AS NumberOfInvoicesForProduct,
    SUM(CASE WHEN c.Quantity > 0 THEN 1 ELSE 0 END) AS NumberOfPositiveItemLinesForProduct,
    ROUND(
        CAST(SUM(c.Quantity) AS REAL) /
        SUM(SUM(c.Quantity)) OVER (PARTITION BY c.Description),
        2)
    AS PercentageOfProductQuantity
FROM
    data AS c
WHERE
    c.Quantity > 0
    AND c.Customer_ID IS NOT NULL
    AND c.Invoice NOT LIKE 'C%' -- This line directly excludes canceled transactions
GROUP BY
    c.Customer_ID, c.Country, c.Description -- Grouping by Description now
HAVING
    COUNT(DISTINCT c.Invoice) > 1 -- Still defining loyalty as more than one invoice for this specific product
ORDER BY
    TotalQuantityBoughtOfProduct DESC, NumberOfInvoicesForProduct DESC, c.Customer_ID, c.Description;
""", globals())


,Customer_ID,Country,Description,TotalQuantityBoughtOfProduct,NumberOfInvoicesForProduct,NumberOfPositiveItemLinesForProduct,PercentageOfProductQuantity
0,13902.0,Denmark,BLACK AND WHITE PAISLEY FLOWER MUG,25164,2,2,0.98
1,15838.0,United Kingdom,BROCADE RING PURSE,23760,4,5,0.38
2,16754.0,United Kingdom,WORLD WAR 2 GLIDERS ASSTD DESIGNS,23040,10,11,0.25
3,16422.0,United Kingdom,BROCADE RING PURSE,20140,27,28,0.32
4,17940.0,United Kingdom,PACK OF 12 SUKI TISSUES,17960,5,6,0.70
...,...,...,...,...,...,...,...
121026,18283.0,United Kingdom,RED SPOTTY PEG BAG,2,2,2,0.00
121027,18283.0,United Kingdom,RETROSPOT TEA SET CERAMIC 11 PC,2,2,2,0.00
121028,18283.0,United Kingdom,ROUND SNACK BOXES SET OF4 WOODLAND,2,2,2,0.00
121029,18283.0,United Kingdom,SET OF 3 BUTTERFLY COOKIE CUTTERS,2,2,2,0.00


This query returns **aggregated summaries of purchasing behavior** for specific customers, rather than individual transactions.

Each row in the result set represents a **unique combination of a customer, their country, and a specific product description (item)** that meets certain criteria.

Let's break down what defines each row in the output:

1. Who is included?

    - Only identifiable customers (Customer_ID IS NOT NULL).
    - Only customers involved in actual sales (where Quantity is positive).
    - Only customers involved in non-canceled orders (where Invoice does NOT start with 'C').



2. What constitutes a row?

    - Each row is a summary for a specific Customer_ID, from a particular Country, and concerning one Description (i.e., one distinct product item).



3. What makes them "loyal" for this specific product?

    - The HAVING COUNT(DISTINCT c.Invoice) > 1 clause is key here. It filters the aggregated rows to only include those where the customer has purchased that particular Description (product item) on more than one distinct, non-canceled invoice. This is how the query defines "loyal" in this context – a customer isn't just a one-time buyer of that specific item.



4. What information do you get for each row?

    - ```Customer_ID```: The identifier of the customer.
    - ```Country```: The country of that customer.
    - ```Description```: The specific product item (e.g., "WHITE HANGING HEART T-LIGHT HOLDER", "REGENCY CAKESTAND 3 TIER") that the customer has repeatedly purchased.
    - ```TotalQuantityBoughtOfProduct```: The sum of all positive quantities of that specific product purchased by that customer across all their relevant invoices.
    - ```NumberOfInvoicesForProduct```: The count of distinct invoices where this customer purchased that specific product. (This value will always be greater than 1 due to the ```HAVING``` clause).
    - ```NumberOfPositiveItemLinesForProduct```: The total count of individual line items (product entries within an invoice) where the quantity was positive, for that specific customer and product.
    - ```PercentageOfProductQuantity```: This shows how much this specific customer's ```TotalQuantityBoughtOfProduct``` contributes to the overall total quantity sold of that specific product (Description) across all customers in the dataset.



**In essence, each row in the result tells you:**

"For this specific customer (from this country), who has bought this particular product more than once, here's the total quantity they bought of it, how many times they bought it, and what percentage of that product's overall sales they represent."